In [2]:
import random

def load_data_set(filename):
    data_mat = []
    label_mat = []
    fr = open(filename)
    for line in fr.readlines():
        line_arr = line.strip().split("\t")
        data_mat.append([float(line_arr[0]), float(line_arr[1])])
        label_mat.append(float(line_arr[2]))
    return data_mat, label_mat

'''
i  alpha的下标
   m是所有alpha的数目
'''
def select_jrand(i, m):
    j = i
    while(j == i):
        j = int(random.uniform(0, m))
    return j

'''
调整alpha 使该值在H和L之间
'''
def clip_alpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

data_arr, label_arr = load_data_set("testSet.txt")

In [3]:
import numpy as np

'''
@dataMat    ：数据列表   
@classLabels：标签列表
@C          ：权衡因子（增加松弛因子而在目标优化函数中引入了惩罚项）
@toler      ：容错率
@maxIter    ：最大迭代次数
'''
def smo_simple(data_matrix_in, class_labels, C, toler, max_iter):
    data_matrix = np.mat(data_matrix_in)
    label_mat = np.mat(class_labels).T
    b = 0
    m, n = np.shape(data_matrix)
    alphas = np.mat(np.zeros((m,1)))
    iter = 0
    while(iter < max_iter):
        alpha_pairs_changed = 0
        for i in range(m):
            # 预测值
            fXi = float(np.multiply(alphas, label_mat).T * (data_matrix*data_matrix[i,:].T)) + b
            Ei = fXi - float(label_mat[i])
            if (label_mat[i]*Ei < -toler and alphas[i] < C) or (label_mat[i]*Ei > toler and alphas[i] > 0):
                j = select_jrand(i, m)  #随机选出第一个alpha的下标
                fXj = float(np.multiply(alphas, label_mat).T * (data_matrix*data_matrix[j,:].T)) + b
                Ej = fXj - float(label_mat[j])
                alpha_i_old = alphas[i].copy()
                alpha_j_old = alphas[j].copy()
                if label_mat[i] != label_mat[j]: #yi!=yj
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L == H:
#                     print("L == H")
                    continue
    # K11+K22-2K12
                eta = 2.0*data_matrix[i,:]*data_matrix[j,:].T - data_matrix[i,:]*data_matrix[i,:].T \
                        - data_matrix[j,:]*data_matrix[j,:].T
                if eta >= 0:
#                     print("eta >= 0")
                    continue
                alphas[j] -= label_mat[j]*(Ei-Ej)/eta
                alphas[j] = clip_alpha(alphas[j], H, L)
                if abs(alphas[j] - alpha_j_old) < 0.00001: #修改值达到0.00001
#                     print("j not moving enough")
                    continue
                alphas[i] += label_mat[j]*label_mat[i]*(alpha_j_old - alphas[j])
                b1 = b - Ei - label_mat[i]*(alphas[i] - alpha_i_old)*data_matrix[i,:]*data_matrix[i,:].T - \
                    label_mat[j]*(alphas[j]-alpha_j_old)*data_matrix[i,:]*data_matrix[j,:].T
                b2 = b - Ej - label_mat[i]*(alphas[i] - alpha_i_old)*data_matrix[i,:]*data_matrix[j,:].T - \
                    label_mat[j]*(alphas[j]-alpha_j_old)*data_matrix[j,:]*data_matrix[j,:].T
                if alphas[i] > 0 and alphas[i] < C:
                    b = b1
                elif alphas[j] > 0 and alphas[j] < C:
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                alpha_pairs_changed += 1
#                 print("iter: %d i: %d, pairs changed %d" % (iter, i , alpha_pairs_changed))
        if alpha_pairs_changed == 0:
            iter += 1
        else:
            iter = 0
#         print("iteration number: %d" % iter)
    return b, alphas

b, alphas = smo_simple(data_arr, label_arr, 0.6, 0.001, 40)
# print(np.shape(alphas[alphas>0]))
print(b)

# for i in range(100):
#     if alphas[i] >0.0:
#         print(data_arr[i], label_arr[i])

[[-3.84100325]]


In [7]:
# a = np.mat([1,2,3])
# b = np.mat([1,2,3])
# print(np.multiply(a, b))



[[1 4 9]]


In [5]:
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = np.shape(dataMatIn)[0]
        self.alphas = np.mat(np.zeros((self.m, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.m, 2)))
    
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas, oS.labelMat).T*(oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 返回最大步长的j，和Ej
def selectJ(i, oS, Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    oS.eCache[i] = [1, Ei]
    validEcacheList = np.nonzero(oS.eCache[:, 0].A)[0]
    if (len(validEchacheList)) > 1:
        for k in validEacheList:
            if k == i: continue
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:
        j = select_jrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

''' 
i  alpha的下标
m是所有alpha的数目
'''
def select_jrand(i, m):
    j = i
    while(j == i):
        j = int(random.uniform(0, m))
    return j

def updateEk(oS, k):
    Ek = calcE(oS, k)
    oS.eCache[k] = [1, Ek]

def innerL(i, os):
    Ei = calcEk(oS, i)
    if (oS.labelMat[i]*Ei < -oS.tol and oS.alphas[i] < oS.C) or\
           (oS.labelMat[i]*Ei > oS.tol and oS.alphas[i] > 0):
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = os.alphas[j].copy()
        if os.labelMat[i] != os.labelMat[j]:
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - os.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, os.alphas[j] + oS.alphas[i])
        if L == H:
            print("L == H")
            return 0
        eta = 2.0*oS.x[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - \
            oS.X[j,:]*oS.X[j,:].T
        if eta >= 0:
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] -= clipAlpha(os.alphas[j])
        updateEk(oS, j)
        if abs(oS.alphas[j] - alphasJold) < 0.00001:
            print("j not moving enough")
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        updateEk(oS, j)
        b1 = oS.b - Ei - oS.labelMat[i]*(oS.alpahas[i] - oS.alphasIold)*\
            oS.X[i,:]*oS.X[i,:].T - oS.label[j]*(oS.alpahas[j] - oS.alphasJold)*oS.X[j,:]*oS.X[i,:].T
        b1 = oS.b - Ej - oS.labelMat[i]*(oS.alpahas[i] - oS.alphasIold)*\
            oS.X[i,:]*oS.X[j,:].T - oS.label[j]*(oS.alpahas[j] - oS.alphasJold)*oS.X[j,:]*oS.X[j,:].T
        # 如果alphas[i]和alphas[j]值在0和C之间，则有b1==b2
        if 0 < oS.alphas[i] < oS.C:
            oS.b = b1
        elif 0 < oS.alphas[j] < oS.C:
            oS.b = b2
        else:
            os.b = (b1+b2)/2
        return 1
    else:
        return 0

def smop(dataMat)